# Edgar Filings - Financials

This notebook shows how to use Algoseek's Edgar Filings dataset to extract financial statements from 10-K and 10-Q filings. The notebook shows how to extract a list of all available financial statements for a given company and how to extract the content of a specific financial statement.

To start, we need to create a connection to the Algoseek Edgar Filings database. The service can be accessed using the `clickhouse_connect` library. The connection parameters are provided by Algoseek.


In [1]:
import os
import pandas as pd
from IPython.display import HTML, display
import clickhouse_connect
from dotenv import load_dotenv

load_dotenv()

client = clickhouse_connect.get_client(
    host=os.environ["CLICKHOUSE_HOST"],
    port=int(os.environ["CLICKHOUSE_PORT"]),
    user=os.environ["CLICKHOUSE_USER"],
    password=os.environ["CLICKHOUSE_PASSWORD"],
    database=os.environ["CLICKHOUSE_DATABASE"],
)


## List all 10-K and 10-Q filings for a given ticker

for this example we will list the latest 10 filings for the ticker `AMZN`.

### The query

In [13]:
database = os.environ["CLICKHOUSE_DATABASE"]

ticker = "AMZN"

query = f"""
    SELECT ReportId, AccessionNumber::String, PeriodEndDate::Date, DocumentType
    FROM {database}.reports r
    JOIN {database}.entities e ON r.EntityId = e.EntityId
    WHERE e.Ticker = '{ticker}' AND r.DocumentType IN ('10-K', '10-Q')
    ORDER BY PeriodEndDate DESC
    LIMIT 10
    """



    SELECT ReportId, AccessionNumber::String, PeriodEndDate::Date, DocumentType
    FROM EdgarFilingsBeta.reports r
    JOIN EdgarFilingsBeta.entities e ON r.EntityId = e.EntityId
    WHERE e.Ticker = 'AMZN' AND r.DocumentType IN ('10-K', '10-Q')
    ORDER BY PeriodEndDate DESC
    LIMIT 10
    


### Response

Now we fetch the results from the database, and display the response.

In [14]:
result_filings_list = client.query(query)
# Convert the result to a pandas DataFrame
df = pd.DataFrame(
    result_filings_list.result_rows, columns=result_filings_list.column_names
)

# Display the DataFrame
display(HTML(df.to_html(notebook=True)))

,ReportId,"CAST(AccessionNumber, 'String')","CAST(PeriodEndDate, 'Date')",DocumentType
0,306093733,0001018724-24-000083,2024-03-31,10-Q
1,306080234,0001018724-24-000008,2023-12-31,10-K
2,306051538,0001018724-23-000018,2023-09-30,10-Q
3,305992881,0001018724-23-000012,2023-06-30,10-Q
4,305960174,0001018724-23-000008,2023-03-31,10-Q
5,305923291,0001018724-23-000004,2022-12-31,10-K
6,305380753,0001018724-22-000023,2022-09-30,10-Q
7,305388743,0001018724-22-000019,2022-06-30,10-Q
8,305351888,0001018724-22-000013,2022-03-31,10-Q
9,305270732,0001018724-22-000005,2021-12-31,10-K


## Display a single financial statement for a given filing

For this example, we will list the balance sheet for the report with accession number `0001018724-24-000008`. This is the `2023-12-31` `10-K` report for Amazon.com, Inc. (`AMZN`) mentioned in the previous example.

Note the ordering statements. The `network_role_description`, `relationship_tree_sequence`, and `relationship_tree_depth` fields in the `financials` table will help you build a tree-like structure of the filing.

### The query

In [26]:
first_record = df.iloc[0]['ReportId']

query_filing = f"""
        SELECT
            f.TreeSequence,
            f.TreeDepth,
            f.FiscalYear,
            f.FiscalPeriod,
            f.Network,
            f.ConceptLabel,
            f.Value
        FROM {database}.financials f
        WHERE f.ReportId = '{first_record}'
    """


### Response

Finally, we fetch the results from the database and display them.


In [28]:
result_filing = client.query(query_filing)

# Convert the result to a pandas DataFrame
df = pd.DataFrame(result_filing.result_rows, columns=result_filing.column_names)

# Display the DataFrame
display(HTML(df.to_html(notebook=True)))

,TreeSequence,TreeDepth,FiscalYear,FiscalPeriod,Network,ConceptLabel,Value
0,1,1,2024.0,1Q,0000001 - Document - Cover,Document Type,10-Q
1,2,1,2024.0,1Q,0000001 - Document - Cover,Document Quarterly Report,true
2,3,1,2024.0,1Q,0000001 - Document - Cover,Document Period End Date,2024-03-31
3,4,1,2024.0,1Q,0000001 - Document - Cover,Document Transition Report,false
4,5,1,2024.0,1Q,0000001 - Document - Cover,Entity File Number,000-22513
5,6,1,2024.0,1Q,0000001 - Document - Cover,Entity Registrant Name,"AMAZON.COM, INC."
6,7,1,2024.0,1Q,0000001 - Document - Cover,"Entity Incorporation, State or Country Code",DE
7,8,1,2024.0,1Q,0000001 - Document - Cover,Entity Tax Identification Number,91-1646860
8,9,1,2024.0,1Q,0000001 - Document - Cover,"Entity Address, Address Line One",410 Terry Avenue North
9,10,1,2024.0,1Q,0000001 - Document - Cover,"Entity Address, City or Town","Seattle,"
